In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModel
from peft import PeftModel
import time
import re

C:\Users\aatan\anaconda3\envs\pytorch_pkg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# # openai-community/roberta-large-openai-detector

# class DetectingLLM(nn.Module):
#     def __init__(self, model_name="Jahid05/llama-3.2-1b-text-classification"):
#         super(DetectingLLM, self).__init__()

#         # Load tokenizer
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)

#         # Load base model (without classification head)
#         self.encoder = AutoModel.from_pretrained(model_name)
#         hidden_size = self.encoder.config.hidden_size

#         # Classification head (you can customize dimensions)
#         self.classifier = nn.Sequential(
#             nn.Linear(hidden_size, 128),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(128, 2)  # For binary classification
#         )

#     def forward(self, text_input, device):
#         # Tokenize
#         inputs = self.tokenizer(
#             text_input,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=256
#         ).to(device)

#         # Forward pass through LLM
#         outputs = self.encoder(**inputs)
#         cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] token

#         # Classification
#         logits = self.classifier(cls_embedding)
#         return logits

In [4]:
class DetectingLLM(nn.Module):
    def __init__(self, model_name="FacebookAI/roberta-large-mnli"):
        super(DetectingLLM, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2,
            ignore_mismatched_sizes=True  # <--- This avoids the size mismatch error
        )
        self.model.resize_token_embeddings(len(self.tokenizer))  # <--- Ensures vocab sizes match

        for param in self.model.base_model.parameters():
            param.requires_grad = False

    def forward(self, text_input, device):
        inputs = self.tokenizer(
            text_input,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = self.model(**inputs)
        return outputs.logits

In [5]:
# Initialize Model, Optimizer, and Loss Function
model = DetectingLLM().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

C:\Users\aatan\anaconda3\envs\pytorch_pkg\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aatan\.cache\huggingface\hub\models--FacebookAI--roberta-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initiali

In [6]:
# Custom Dataset for Loading Text Data
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [7]:
# Load and preprocess the dataset
df = pd.read_csv("dataset/gemma_marked_7k.csv")

In [8]:
df.shape

(14000, 2)

In [9]:
# Clean and format the text
def clean_text(raw_text):
    # Remove extra newlines and whitespace
    cleaned_text = re.sub(r'\s*\n\s*', '\n', raw_text.strip())
    
    # Remove unnecessary quotes
    cleaned_text = cleaned_text.strip("'\"")
    
    # Replace escaped newline characters with actual newlines
    cleaned_text = cleaned_text.replace("\\n", "\n")
    cleaned_text = cleaned_text.replace("**", "")
    cleaned_text = cleaned_text.replace("\n", ". ")
    cleaned_text = cleaned_text.replace("## ", "")
    
    # Remove excessive blank lines
    cleaned_text = re.sub(r'\n{2,}', '\n\n', cleaned_text)
    
    # Ensure proper formatting
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [10]:
df['text'] = df['text'].apply(clean_text)

In [11]:
df.head()

,text,label
0,"Sure, here's an explanation of the importance ...",0
1,Academic writing can help to solve issues in c...,0
2,Vocational training can help to solve issues i...,0
3,Strategies for Improving Adult Education in Ed...,0
4,Inclusive Education and Digital Textbooks. 1. ...,0


In [12]:
df["label"] = df["label"].astype(int)

In [13]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [14]:
shuffled_df.head()

,text,label
0,Unique Perspectives on Allegory in Satire:. 1....,0
1,Cultural Relativism and Theories of Happiness....,0
2,Embedding Watermarks in Biomedical Engineering...,1
3,Argument for Thought Experiments in Philosophy...,0
4,Nanotechnology in Particle Physics. Nanotechno...,1


In [18]:
# Split into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    shuffled_df['text'], shuffled_df['label'], test_size=0.2, random_state=42
)

# Create Datasets and DataLoaders
train_dataset = TextDataset(train_texts.tolist(), train_labels.tolist())
test_dataset = TextDataset(test_texts.tolist(), test_labels.tolist())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [19]:
train_texts.shape

(11200,)

In [20]:
# Evaluation Loop
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for texts, labels in test_loader:
            # Move labels to GPU
            labels = torch.tensor(labels).to(device)
            # Forward pass with inputs on GPU
            outputs = model(texts, device=device)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds)
            actuals.extend(labels.cpu().numpy())
    acc = accuracy_score(actuals, predictions)
    print(f"Accuracy: {acc * 100:.2f}%")

In [21]:
# Training Loop
def train_model(model, train_loader, optimizer, criterion, device, epochs):
    model.train()
    for epoch in range(epochs):  # Train for `epochs`
        start = time.time()
        total_loss = 0
        for texts, labels in train_loader:
            # Move labels to GPU
            labels = torch.tensor(labels).to(device)
            optimizer.zero_grad()
            # Forward pass with inputs on GPU
            outputs = model(texts, device=device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        end = time.time()
        print(f"Epoch {epoch + 1}, Time: {end-start:.4f}, Loss: {total_loss / len(train_loader):.4f}")
        #evaluate_model(model, train_loader, device)
        if (epoch+1)%10==0:
            evaluate_model(model, test_loader, device)

In [22]:
train_model(model, train_loader, optimizer, criterion, device, epochs=200)

C:\Users\aatan\AppData\Local\Temp\ipykernel_19796\668303449.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Epoch 1, Time: 43.9376, Loss: 0.6529
Epoch 2, Time: 42.2463, Loss: 0.6029
Epoch 3, Time: 42.6121, Loss: 0.5924
Epoch 4, Time: 43.5618, Loss: 0.5865
Epoch 5, Time: 42.0985, Loss: 0.5796
Epoch 6, Time: 42.4137, Loss: 0.5714
Epoch 7, Time: 41.3255, Loss: 0.5703
Epoch 8, Time: 41.3691, Loss: 0.5652
Epoch 9, Time: 42.0793, Loss: 0.5632
Epoch 10, Time: 42.6261, Loss: 0.5611


C:\Users\aatan\AppData\Local\Temp\ipykernel_19796\374512516.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Accuracy: 74.29%


C:\Users\aatan\AppData\Local\Temp\ipykernel_19796\668303449.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


...

Accuracy: 75.79%


C:\Users\aatan\AppData\Local\Temp\ipykernel_19796\668303449.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Epoch 191, Time: 39.7570, Loss: 0.1482
Epoch 192, Time: 39.8716, Loss: 0.1494
Epoch 193, Time: 40.7772, Loss: 0.1482
Epoch 194, Time: 41.1884, Loss: 0.1398
Epoch 195, Time: 40.6372, Loss: 0.1418
Epoch 196, Time: 40.7364, Loss: 0.1355
Epoch 197, Time: 41.5181, Loss: 0.1395
Epoch 198, Time: 40.6588, Loss: 0.1341
Epoch 199, Time: 41.4072, Loss: 0.1358
Epoch 200, Time: 41.4012, Loss: 0.1321


C:\Users\aatan\AppData\Local\Temp\ipykernel_19796\374512516.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Accuracy: 72.25%


In [ ]:
# Evaluation Loop
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for texts, labels in test_loader:
            # Move labels to GPU
            labels = torch.tensor(labels).to(device)
            # Forward pass with inputs on GPU
            outputs = model(texts, device=device)
            #print(f"outputs: {outputs}")
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds)
            actuals.extend(labels.cpu().numpy())
    # print(f"Actual: {actuals}")
    # print(f"Predictions: {predictions}")
    acc = accuracy_score(actuals, predictions)
    print(f"Test Accuracy: {acc * 100:.2f}%")

In [ ]:
evaluate_model(model, train_loader, device)

In [ ]:
# Get one batch from the test loader
texts, labels = next(iter(test_loader))

# Print the data
print("Texts:", texts)
print("Labels:", labels)

In [ ]:
outputs = model(texts, device=device)

In [ ]:
outputs

In [ ]:
preds = torch.argmax(outputs, dim=1).cpu().numpy()
preds

In [ ]:
evaluate_model(model, train_loader, device)

In [ ]:
# Save the entire model
torch.save(model, "detecting_llm_full_v2.pth")
print("Entire model saved successfully!")

In [ ]:
# Load the entire model
model = torch.load("detecting_llm_full_v2.pth")
model.to(device)  # Move model to the appropriate device
print("Entire model loaded successfully!")